In [ ]:
#load in packages 
import numpy as np
import pandas as pd

#regex
import re

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering

from scipy.cluster.hierarchy import dendrogram
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

import matplotlib.pyplot as plt


In [260]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import words

In [265]:
import nltk

nltk.download('punkt')        # For word_tokenize
nltk.download('stopwords')    # For stopwords
nltk.download('wordnet')      # For WordNetLemmatizer
nltk.download('words')        # For nltk.corpus.words


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jarem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jarem\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jarem\AppData\Roaming\nltk_data...
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jarem\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [174]:
items_df = pd.read_csv("data/input/items_2025.csv")
prices_df = pd.read_csv("data/input/prices_2025_cross_section.csv")

In [175]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117580 entries, 0 to 117579
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   store_id            117580 non-null  int64 
 1   product_id          117580 non-null  int64 
 2   product_title       117580 non-null  object
 3   product_url         117580 non-null  object
 4   product_breadcrumb  117580 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.5+ MB


In [176]:
items_df.shape

(117580, 5)

In [177]:
items_df.head(20)

,store_id,product_id,product_title,product_url,product_breadcrumb
0,3,910001819336,Ella's Kitchen Organic Strawberry and Banana Melty Puffs Baby Snack 6+ Months,http://groceries.asda.com/product/baby-toddler--kids-snacks/ellas-kitchen-organic-strawberry-and-banana-melty-puffs-baby-snack--months/910001819336,"Baby, Toddler & Kids, Baby, Toddler & Kids Snacks, Baby Snacks (Under 12 months), Baby Snacks (Under 12 Months)"
1,3,910001818776,Ella's Kitchen Organic Tomato and Leek Melty Puffs Baby Snack 6+ Months,http://groceries.asda.com/product/baby-toddler--kids-snacks/ellas-kitchen-organic-tomato-and-leek-melty-puffs-baby-snack--months/910001818776,"Baby, Toddler & Kids, Baby, Toddler & Kids Snacks, Baby Snacks (Under 12 months), Baby Snacks (Under 12 Months)"
2,3,910001817408,Regal Bakery Classic Fairy Cakes,http://groceries.asda.com/product/indian--asian-food/regal-bakery-classic-fairy-cakes/910001817408,"World Food, Indian & Asian Food, Asian Desserts & Sweet Treats, Asian Desserts & Sweet Treats"
3,3,910001817566,Innocent Summer Fruits Juice,http://groceries.asda.com/product/chilled-juice--smoothies/innocent-summer-fruits-juice/910001817566,"['Chilled Food', 'Chilled Juice & Smoothies', 'Chilled Juice', 'Berry & Cherry Juice']"
4,3,910001821997,Bulldog Skincare for Men Original Face Wash,http://groceries.asda.com/product/mens-toiletries/bulldog-skincare-for-men-original-face-wash/910001821997,"Toiletries & Beauty, Men's Toiletries, Men's Skin Care, Men's Face Wash & Scrubs"
5,3,910001823893,J2O Spritz Pear & Raspberry Sparkling Juice Drink,http://groceries.asda.com/product/tonic-water--mixers/jo-spritz-pear--raspberry-sparkling-juice-drink/910001823893,"Drinks, Tonic Water & Mixers, Adult Soft Drinks, Adult Soft Drinks"
6,3,910001820433,Copella Cloudy Apple Fruit Juice,http://groceries.asda.com/product/sandwiches-snacks--drinks/copella-cloudy-apple-fruit-juice/910001820433,"Chilled Food, Sandwiches, Snacks & Drinks, Sandwiches, Snacks & Drinks, Drinks"
7,3,910001816585,Nando's Peri-Peri Sauce Hot,http://groceries.asda.com/product/condiments--cooking-ingredients/nandos-periperi-sauce-hot/910001816585,"Food Cupboard, Condiments & Cooking Ingredients, Popular Brands, Nando's"
8,3,910001816627,ASDA Extra Special Chardonnay,http://groceries.asda.com/product/wine/asda-extra-special-chardonnay/910001816627,"Beer, Wine & Spirits, Wine, White Wine, Chardonnay"
9,3,910001816360,ASDA Extra Special Soave Classico,http://groceries.asda.com/product/wine/asda-extra-special-soave-classico/910001816360,"Beer, Wine & Spirits, Wine, White Wine, All Other Grapes"


In [178]:
items_df["product_breadcrumb"]

0                                                                                  Baby, Toddler & Kids, Baby, Toddler & Kids Snacks, Baby Snacks (Under 12 months), Baby Snacks (Under 12 Months)
1                                                                                  Baby, Toddler & Kids, Baby, Toddler & Kids Snacks, Baby Snacks (Under 12 months), Baby Snacks (Under 12 Months)
2                                                                                                    World Food, Indian & Asian Food, Asian Desserts & Sweet Treats, Asian Desserts & Sweet Treats
3                                                                                                           ['Chilled Food', 'Chilled Juice & Smoothies', 'Chilled Juice', 'Berry & Cherry Juice']
4                                                                                                                 Toiletries & Beauty, Men's Toiletries, Men's Skin Care, Men's Face Wash & Scrubs
                         

In [179]:
prices_df.head(15)

,date,store_id,product_id,price,unit_price,loyalty_price,original_price
0,2025-01-01,1,3.253248e+08,4.99,4.99 / each,NaN,NaN
1,2025-01-01,3,1.000383e+12,11.00,£11.00/each,NaN,NaN
2,2025-01-01,2,7.621751e+06,1.80,£8.57 / kg,NaN,1.8
3,2025-01-01,1,3.252527e+08,144.99,144.99 / each,NaN,NaN
4,2025-01-01,2,8.163367e+06,1.30,£1.30 / 100g,NaN,1.3
5,2025-01-01,2,8.201977e+06,1.90,£7.60 / kg,NaN,1.9
6,2025-01-01,1,3.165419e+08,8.50,36.96 / kg,NaN,NaN
7,2025-01-01,1,3.250867e+08,14.99,14.99 / each,NaN,NaN
8,2025-01-01,2,8.199674e+06,1.30,£1.44 / 100g,NaN,1.3
9,2025-01-01,1,3.253245e+08,6.99,6.99 / each,NaN,NaN


In [180]:
prices_df["date"]

0         2025-01-01
1         2025-01-01
2         2025-01-01
3         2025-01-01
4         2025-01-01
             ...    
117548    2025-02-12
117549    2025-02-12
117550    2025-02-12
117551    2025-02-12
117552    2025-02-12
Name: date, Length: 117553, dtype: object

In [181]:
#prices_df = prices_df.set_index("product_id")
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117553 entries, 0 to 117552
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            117553 non-null  object 
 1   store_id        117553 non-null  int64  
 2   product_id      117553 non-null  float64
 3   price           117553 non-null  float64
 4   unit_price      93485 non-null   object 
 5   loyalty_price   7475 non-null    float64
 6   original_price  24971 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 6.3+ MB


In [182]:
print(list(prices_df["unit_price"].unique()))

['4.99 / each', '£11.00/each', '£8.57 / kg', '144.99 / each', '£1.30 / 100g', '£7.60 / kg', '36.96 / kg', '14.99 / each', '£1.44 / 100g', '6.99 / each', '£12.00/each', '27.49 / each', '61.99 / each', '79.99 / each', '8.57 / kg', '3.43 / 100g', '9.99 / each', '25.19 / each', '£17.71/kg', '£10.00/each', '19.99 / each', '75.0p/each', '5.99 / each', '29.99 / each', '44.99 / each', '26.69 / each', '1.5 / 100ml', '6.59 / each', '8.99 / each', '84.99 / each', '49.99 / each', '10.99 / each', '38.99 / each', '9.49 / each', '30 / each', '12.99 / each', '34.99 / each', '21.99 / each', '13.99 / each', '139 / each', '27.99 / each', '16 / each', '16.99 / each', '7.99 / each', '22.99 / each', '20.99 / each', '15.99 / each', '17.99 / each', '11.99 / each', '97.6p/100g', '24.99 / each', '71.99 / each', '25.49 / each', '9.69 / each', '39.99 / each', '17.49 / each', '3.99 / each', '2.5 / 100g', '105.99 / each', '26.99 / each', '28.99 / each', '35.99 / each', '99.99 / each', '10.79 / each', '74.99 / each'

In [183]:
#prices_df = prices_df.dropna(subset = ["loyalty_price"])
#prices_df = prices_df.dropna(subset=["original_price"])

In [212]:
# match datasets
merged_df = items_df.copy()

In [213]:
merged_df = merged_df.merge(prices_df, how='right', on = "product_id", copy=False)
merged_df["store_id"] = merged_df["store_id_x"]
merged_df = merged_df.drop(columns=["store_id_y", "store_id_x"])
merged_df = merged_df.set_index("product_id")

In [214]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117581 entries, 325324827 to 4088600084398
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   product_title       117581 non-null  object 
 1   product_url         117581 non-null  object 
 2   product_breadcrumb  117581 non-null  object 
 3   date                117581 non-null  object 
 4   price               117581 non-null  float64
 5   unit_price          93513 non-null   object 
 6   loyalty_price       7475 non-null    float64
 7   original_price      24979 non-null   float64
 8   store_id            117581 non-null  int64  
dtypes: float64(3), int64(1), object(5)
memory usage: 9.0+ MB


## Clean up breadcrumb column

In [187]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# clean the breadcrumb string
def clean_breadcrumb(breadcrumb):
    breadcrumb = breadcrumb.replace("'", "").replace("en-GB", "").replace("[","").replace("]","").replace('"','')  # Remove unwanted characters
    breadcrumb = breadcrumb.replace(", ", " > ").replace("/", " > ").replace("-", " ")  # Replace characters

    if breadcrumb.startswith("Marketplace"):
        breadcrumb = breadcrumb.replace("Marketplace >", "", 1)
    
    elif breadcrumb.startswith("Home"):
        breadcrumb = breadcrumb.replace("Home","",1)
    
    elif breadcrumb.startswith("Groceries"):
        breadcrumb = breadcrumb.replace("Groceries","",1)
    return breadcrumb.strip() 

In [189]:
# Apply cleanup to the 'product_breadcrumb' column
#merged_df["product_breadcrumb"] = merged_df["product_breadcrumb"].apply(clean_breadcrumb)

In [217]:
merged_df["product_breadcrumb"] = merged_df["product_breadcrumb"].str.replace(r"^>+\s*", "", regex=True)

In [218]:
merged_df["text"] = merged_df["product_breadcrumb"] + " - " + merged_df["product_title"]

In [219]:
merged_df.iloc[2342:2354,:]

,product_title,product_url,product_breadcrumb,date,price,unit_price,loyalty_price,original_price,store_id,text
product_id,,,,,,,,,,
113941940,Morrisons White & Green Foliage Dinner Crackers,https://groceries.morrisons.com/products/morrisons-white--green-foliage-dinner-crackers/113941940,Toys & Entertainment > Gift Shop,2025-01-10,6.00,NaN,NaN,NaN,4,Toys & Entertainment > Gift Shop - Morrisons White & Green Foliage Dinner Crackers
113933632,Morrisons Santa Boppers Headband,https://groceries.morrisons.com/products/morrisons-santa-boppers-headband/113933632,Toys & Entertainment > Gift Shop,2025-01-10,1.00,NaN,NaN,NaN,4,Toys & Entertainment > Gift Shop - Morrisons Santa Boppers Headband
113941915,Morrisons Robin Dinner Crackers,https://groceries.morrisons.com/products/morrisons-robin-dinner-crackers/113941915,Toys & Entertainment > Gift Shop,2025-01-10,6.00,NaN,NaN,NaN,4,Toys & Entertainment > Gift Shop - Morrisons Robin Dinner Crackers
113284185,Barbie Design Your Own Drink Bottle,https://groceries.morrisons.com/products/barbie-design-your-own-drink-bottle/113284185,Toys & Entertainment > Christmas Clearance > Gifting & Toys,2025-01-10,6.00,NaN,NaN,NaN,4,Toys & Entertainment > Christmas Clearance > Gifting & Toys - Barbie Design Your Own Drink Bottle
114217048,Funkin Black Forest Gateau,https://groceries.morrisons.com/products/funkin-black-forest-gateau/114217048,"Beer, Wines & Spirits > Pre Mixed Drinks & Cocktails",2025-01-10,12.50,NaN,NaN,NaN,4,"Beer, Wines & Spirits > Pre Mixed Drinks & Cocktails - Funkin Black Forest Gateau"
113977627,Fizz Creations Make Your Own White Neon Effect Sign,https://groceries.morrisons.com/products/fizz-creations--make-your-own-white-neon-effect-sign/113977627,Toys & Entertainment > Gift Shop,2025-01-10,6.00,NaN,NaN,NaN,4,Toys & Entertainment > Gift Shop - Fizz Creations Make Your Own White Neon Effect Sign
114148257,Blue Sky Transformers Pen Fidget,https://groceries.morrisons.com/products/blue-sky-transformers-pen-fidget/114148257,Toys & Entertainment > Gift Shop,2025-01-10,4.00,NaN,NaN,NaN,4,Toys & Entertainment > Gift Shop - Blue Sky Transformers Pen Fidget
113979606,Paladone Arcade Keyring,https://groceries.morrisons.com/products/paladone-arcade-keyring/113979606,Toys & Entertainment > Gift Shop,2025-01-10,5.00,NaN,NaN,NaN,4,Toys & Entertainment > Gift Shop - Paladone Arcade Keyring
113979260,Paladone Minecraft Creeper Stressball,https://groceries.morrisons.com/products/paladone-minecraft-creeper-stressball/113979260,Toys & Entertainment > Gift Shop,2025-01-10,5.00,NaN,NaN,NaN,4,Toys & Entertainment > Gift Shop - Paladone Minecraft Creeper Stressball


## Create Data-feature-metrix

In [263]:
# Initialize Snowball Stemmer for English
stemmer = SnowballStemmer("english")

#define a stemmer function
def custom_tokenizer(text):
    return [stemmer.stem(word) for word in text.split()]

#define a preprocessor function
def custom_preprocessor(text):
    
    text = text.lower()  # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [word for word in tokens if word in english_words]  # Remove non-english words
    return " ".join(tokens)


In [264]:
vectorizer = CountVectorizer(ngram_range=(1, 1), preprocessor=custom_preprocessor, tokenizer=custom_tokenizer)

items_vect = vectorizer.fit_transform(merged_df["text"])

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\jarem/nltk_data'
    - 'c:\\Users\\jarem\\AppData\\Local\\Programs\\Python\\Python313\\nltk_data'
    - 'c:\\Users\\jarem\\AppData\\Local\\Programs\\Python\\Python313\\share\\nltk_data'
    - 'c:\\Users\\jarem\\AppData\\Local\\Programs\\Python\\Python313\\lib\\nltk_data'
    - 'C:\\Users\\jarem\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [247]:
items_vect

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1554551 stored elements and shape (117581, 23333)>

In [250]:
items_dfm= pd.DataFrame(
    items_vect.toarray(),
    columns=vectorizer.get_feature_names_out()
)

In [251]:
print(list(items_dfm.columns))

['"00"', '"athlet', '"babi', '"bodi', '"chees', '"children', '"handbag"', '"keepsak', '"kid', '"men', '"our', '"the', '"verstappen', '"view', '"women', '#3147', '#win', '%', '%)', '(', '(0', '(0)', '(1', '(1+', '(1.02kg)', '(1.0mm', '(1.155kg)', '(1.25l', '(1.2kg)', '(1.36kg)', '(1.431', '(1.44', '(1.8m)', '(10', '(10)', '(10+', '(10+)', '(100', '(1000', '(100g)', '(100pk)', '(100x75cm)', '(102*)', '(102g', '(102g)', '(102g*)', '(1030g)', '(103g)', '(104.4g)', '(105g)', '(108g)', '(10miu)', '(10x', '(110*)', '(110g', '(110g)', '(110g*)', '(112', '(112g)', '(112g*)', '(112x76cm)', '(115g)', '(115g*)', '(118.8g)', '(11x40g)', '(12', '(12"', '(12+', '(1200g)', '(120g)', '(120g*)', '(121.2g)', '(123g)', '(124g)', '(125g', '(125g)', '(125g*)', '(126g)', '(127x81cm)', '(128g)', '(129g)', '(12in)', '(12x', '(12x70g)', '(130.9g)', '(130g)', '(130g*)', '(130x170cm)', '(132.8g)', '(132g)', '(133g)', '(135g)', '(136g', '(136g)', '(138g)', '(14', '(140g)', '(140g*)', '(142g)', '(144.9g)', '(144g)'

## Create product categories and subcategories

In [198]:
#Extract product category and subcategory
split_breadcrumbs = merged_df["product_breadcrumb"].str.split(" > ", expand=True)
merged_df["product_category"] = split_breadcrumbs[0]
merged_df["product_subcategory"] = split_breadcrumbs[1]
merged_df["product_subcategory_2"] = split_breadcrumbs[2]

# Convert new columns to lowercase
merged_df["product_breadcrumb"] = merged_df["product_breadcrumb"].str.lower()
merged_df["product_category"] = merged_df["product_category"].str.lower()
merged_df["product_subcategory"] = merged_df["product_subcategory"].str.lower()
merged_df["product_subcategory_2"] = merged_df["product_subcategory_2"].str.lower()


In [199]:
#store id 
n=8
#filter by sore_id
filtered_df = merged_df[merged_df["store_id"] == n]
#randomly choose 20 products to be displayed
random_sample = filtered_df.sample(n=10) #do not set random state!!
random_sample 

,product_title,product_url,product_breadcrumb,date,price,unit_price,loyalty_price,original_price,store_id,text,product_category,product_subcategory,product_subcategory_2
product_id,,,,,,,,,,,,,
899184202551202552,Jacob's Savours Sour Cream & Chive Thins Crackers,https://www.waitrose.com/ecom/products/jacobs-savours-sour-cream--chive-thins-crackers/899184-202551-202552,groceries > food cupboard > biscuits & crackers > crackers & savoury biscuits > crackers > flavoured,2025-02-12,2.00,£1.34/100g,NaN,NaN,8,Groceries > Food Cupboard > Biscuits & Crackers > Crackers & Savoury Biscuits > Crackers > Flavoured - Jacob's Savours Sour Cream & Chive Thins Crackers,groceries,food cupboard,biscuits & crackers
784340756109756110,Discarded Banana Peel Rum,https://www.waitrose.com/ecom/products/discarded-banana-peel-rum/784340-756109-756110,"groceries > beer, wine & spirits > spirits & liqueurs > rum > dark rum",2025-02-12,32.50,£46.43/litre,NaN,NaN,8,"Groceries > Beer, Wine & Spirits > Spirits & Liqueurs > Rum > Dark Rum - Discarded Banana Peel Rum",groceries,"beer, wine & spirits",spirits & liqueurs
412202038820389,Galbani Italian Mozzarella Cheese,https://www.waitrose.com/ecom/products/galbani-italian-mozzarella-cheese/041220-20388-20389,"groceries > fresh & chilled > cheese > mozzarella, mascarpone & ricotta > mozzarella",2025-02-12,2.00,£16/kg,NaN,NaN,8,"Groceries > Fresh & Chilled > Cheese > Mozzarella, Mascarpone & Ricotta > Mozzarella - Galbani Italian Mozzarella Cheese",groceries,fresh & chilled,cheese
402236261097261098,Nando's Medium Peri-Peri Sauce,https://www.waitrose.com/ecom/products/nandos-medium-periperi-sauce/402236-261097-261098,"groceries > food cupboard > condiments, dressings & marinades > hot & chilli sauce",2025-02-12,2.35,£1.88/100g,NaN,NaN,8,"Groceries > Food Cupboard > Condiments, Dressings & Marinades > Hot & Chilli Sauce - Nando's Medium Peri-Peri Sauce",groceries,food cupboard,"condiments, dressings & marinades"
757378741949741950,Starbucks House Blend by Nespresso Coffee Pods 10s,https://www.waitrose.com/ecom/products/starbucks-house-blend-by-nespresso-coffee-pods-10s/757378-741949-741950,"groceries > tea, coffee & soft drinks > coffee > coffee pods > starbucks",2025-02-12,2.95,£5.18/100g,NaN,4.25,8,"Groceries > Tea, Coffee & Soft Drinks > Coffee > Coffee Pods > Starbucks - Starbucks House Blend by Nespresso Coffee Pods 10s",groceries,"tea, coffee & soft drinks",coffee
754693507938507939,Essential Extra Large Whole Chicken,https://www.waitrose.com/ecom/products/essential-extra-large-whole-chicken/754693-507938-507939,groceries > fresh & chilled > fresh chicken & poultry > chicken > whole chicken,2025-02-12,6.79,£2.95/kg,NaN,NaN,8,Groceries > Fresh & Chilled > Fresh Chicken & Poultry > Chicken > Whole Chicken - Essential Extra Large Whole Chicken,groceries,fresh & chilled,fresh chicken & poultry
665003366733668,Sacla' Italia Sun-Dried Tomato Cook's Paste,https://www.waitrose.com/ecom/products/sacla-italia-sundried-tomato-cooks-paste/066500-33667-33668,groceries > food cupboard > cooking sauces & meal kits > seasoning & rubs,2025-02-12,3.50,£1.85/100g,NaN,NaN,8,Groceries > Food Cupboard > Cooking Sauces & Meal Kits > Seasoning & Rubs - Sacla' Italia Sun-Dried Tomato Cook's Paste,groceries,food cupboard,cooking sauces & meal kits
424495494708494709,Piccolo Spring Greens,https://www.waitrose.com/ecom/products/piccolo-spring-greens/424495-494708-494709,"groceries > baby, child & parent > baby & toddler food > 4+ months (stage 1)",2025-02-12,0.97,97p/100g,NaN,1.30,8,"Groceries > Baby, Child & Parent > Baby & Toddler Food > 4+ Months (Stage 1) - Piccolo Spring Greens",groceries,"baby, child & parent",baby & toddler food
504261721965721966,BFree Gluten Free Multigrain Wraps,https://www.waitrose.com/ecom/products/bfree-gluten-free-multigrain-wraps/504261-721965-721966,"groceries > bakery > wraps, tortillas & pitta bread",2025-02-12,3.75,£1.49/100g,NaN,NaN,8,"Groceries > Bakery > Wraps, Tortillas & Pitta Bread - BFree Glu

## Create a filter to keep products of interest

Keep toiletries, household products, kids products 

In [200]:
product_cat_of_interest = ["toiletries", "health", "beauty", "household", "sports" ,"leisure", "laundry", "baby"]

#create a regex pattern to check if any of the strings in list above are contained in rows
pattern = "|".join(product_cat_of_interest)

In [201]:
# Keep only rows that contain any of the categories from the list
filtered_items_df = merged_df[
    merged_df["product_category"].str.contains(pattern, case=False, na=False) |
    merged_df["product_subcategory"].str.contains(pattern, case=False, na=False)
]

In [202]:
print(filtered_items_df.shape)

(25283, 13)


In [203]:
rand_sample_df = filtered_items_df.sample(n=5)
rand_sample_df

,product_title,product_url,product_breadcrumb,date,price,unit_price,loyalty_price,original_price,store_id,text,product_category,product_subcategory,product_subcategory_2
product_id,,,,,,,,,,,,,
1000383224160,Comfort Botanical Fabric Conditioner Summer Bouquet 960 ml (64 washes),http://groceries.asda.com/product/laundry/comfort-botanical-fabric-conditioner-summer-bouquet--ml--washes/1000383224160,"laundry & household, laundry, fabric conditioner & softeners, fabric conditioner & softeners",2025-02-12,5.00,£5.21/lt,NaN,NaN,3,"Laundry & Household, Laundry, Fabric Conditioner & Softeners, Fabric Conditioner & Softeners - Comfort Botanical Fabric Conditioner Summer Bouquet 960 ml (64 washes)","laundry & household, laundry, fabric conditioner & softeners, fabric conditioner & softeners",None,None
316317920,Dove Anti Stress Advanced Care Body Wash Shower Gel Chamomile & Oat Milk 400ml,www.tesco.com/groceries/en-GB/products/316317920,"['health & beauty', 'shower, bath & hand hygiene', 'shower', 'shower gel & body wash']",2025-02-10,4.50,1.12 / 100ml,NaN,NaN,1,"['Health & Beauty', 'Shower, Bath & Hand Hygiene', 'Shower', 'Shower Gel & Body Wash'] - Dove Anti Stress Advanced Care Body Wash Shower Gel Chamomile & Oat Milk 400ml","['health & beauty', 'shower, bath & hand hygiene', 'shower', 'shower gel & body wash']",None,None
108258396,Method Clementine Daily Kitchen Cleaner,https://groceries.morrisons.com/products/method-clementine-daily-kitchen-cleaner/108258396,household > cleaning cupboard > kitchen & oven > sprays,2025-02-12,2.75,NaN,NaN,4.0,4,Household > Cleaning Cupboard > Kitchen & Oven > Sprays - Method Clementine Daily Kitchen Cleaner,household,cleaning cupboard,kitchen & oven
880792730974730975,Garnier Moisture Bomb Tissue Mask,https://www.waitrose.com/ecom/products/garnier-moisture-bomb-tissue-mask/880792-730974-730975,"groceries > toiletries, health & beauty > facial skincare > face masks",2025-02-12,3.70,£13.22/100g,NaN,NaN,8,"Groceries > Toiletries, Health & Beauty > Facial Skincare > Face Masks - Garnier Moisture Bomb Tissue Mask",groceries,"toiletries, health & beauty",facial skincare
114283643,Lynx Fine Fragrance Black Vanilla Duo Gift Set,https://groceries.morrisons.com/products/lynx-fine-fragrance-black-vanilla-duo-gift-set/114283643,toiletries & beauty > gifting & fragrances > gifting,2025-01-16,6.00,NaN,NaN,NaN,4,Toiletries & Beauty > Gifting & Fragrances > Gifting - Lynx Fine Fragrance Black Vanilla Duo Gift Set,toiletries & beauty,gifting & fragrances,gifting


## Assign gender two those products that mention gender in breadcrumb

In [204]:
# Function to classify gender based on the breadcrumb

def classify_gender(breadcrumb):
    # Define regex patterns for male, female, and neutral categories 
    male_pattern = r'\b(male|men|mens|man|mans|masculine|him|for him|his)\b'
    female_pattern = r'\b(feminine|for her|her|hers|women|female|womans|womens|woman|makeup|make up|nailpolish|nail polish)\b' # add makeup and nailpolish as products marketed for women
    
    # Check if the breadcrumb matches the male pattern
    if re.search(male_pattern, breadcrumb, re.IGNORECASE):
        return 'male'
    # Check if the breadcrumb matches the female pattern
    elif re.search(female_pattern, breadcrumb, re.IGNORECASE):
        return 'female'
    # Assign neutral if neither male nor female is found
    else:
        return 'neutral'

In [205]:
df = filtered_items_df.copy()
df.loc[:,'gender_breadcrumb'] = filtered_items_df['product_breadcrumb'].apply(classify_gender)

In [206]:
df.loc[:,'gender_title'] = filtered_items_df['product_title'].apply(classify_gender)

In [207]:
rand_sample_df = df.sample(n=50)
rand_sample_df

,product_title,product_url,product_breadcrumb,date,price,unit_price,loyalty_price,original_price,store_id,text,product_category,product_subcategory,product_subcategory_2,gender_breadcrumb,gender_title
product_id,,,,,,,,,,,,,,,
106182771,Fairy Non Bio For Sensitive Skin Washing Powder 40 Washes,https://groceries.morrisons.com/products/fairy-non-bio-for-sensitive-skin-washing-powder-40-washes/106182771,household > laundry > detergents > washing powder,2025-02-12,9.00,NaN,NaN,NaN,4,Household > Laundry > Detergents > Washing Powder - Fairy Non Bio For Sensitive Skin Washing Powder 40 Washes,household,laundry,detergents,neutral,neutral
113867989,Heinz Mango Chicken And Vegetable Baby Food Jar 7+ Months,https://groceries.morrisons.com/products/heinz-mango-chicken-and-vegetable-baby-food-jar-7-months/113867989,baby & toddler > baby & toddler food & drinks > stage 2 (7-10 months) > main meals,2025-02-12,1.20,NaN,NaN,NaN,4,Baby & Toddler > Baby & Toddler Food & Drinks > Stage 2 (7-10 Months) > Main Meals - Heinz Mango Chicken And Vegetable Baby Food Jar 7+ Months,baby & toddler,baby & toddler food & drinks,stage 2 (7-10 months),neutral,neutral
112555254,Imperial Leather Foamburst Shower Gel Cotton & Vanilla Orchid,https://groceries.morrisons.com/products/imperial-leather-foamburst-shower-gel-cotton--vanilla-orchid/112555254,"toiletries & beauty > bath, shower & soap > shower",2025-02-12,2.50,NaN,NaN,NaN,4,"Toiletries & Beauty > Bath, Shower & Soap > Shower - Imperial Leather Foamburst Shower Gel Cotton & Vanilla Orchid",toiletries & beauty,"bath, shower & soap",shower,neutral,neutral
100696922,Pro Plus Caffeine Tablets,https://groceries.morrisons.com/products/pro-plus-caffeine-tablets/100696922,"health, wellbeing & medicines > vitamins & supplements > energy",2025-02-12,4.00,NaN,NaN,NaN,4,"Health, Wellbeing & Medicines > Vitamins & Supplements > Energy - Pro Plus Caffeine Tablets","health, wellbeing & medicines",vitamins & supplements,energy,neutral,neutral
1513609,John Frieda Frizz-Ease Dream Curls Daily Styling Spray,http://groceries.asda.com/product/hair-care-dye--styling/john-frieda-frizzease-dream-curls-daily-styling-spray/1513609,"toiletries & beauty, hair care, dye & styling, hairspray & styling, texturising sprays & serums",2025-02-12,7.00,£3.50/100ml,NaN,NaN,3,"Toiletries & Beauty, Hair Care, Dye & Styling, Hairspray & Styling, Texturising Sprays & Serums - John Frieda Frizz-Ease Dream Curls Daily Styling Spray","toiletries & beauty, hair care, dye & styling, hairspray & styling, texturising sprays & serums",None,None,neutral,neutral
325116517,skinChemists Advanced Brightening Toner 250ml,www.tesco.com/groceries/en-GB/products/325116517,"['marketplace', 'health & beauty', 'skincare', 'face cleansing']",2025-01-17,14.90,14.9 / each,NaN,NaN,1,"['Marketplace', 'Health & Beauty', 'Skincare', 'Face Cleansing'] - skinChemists Advanced Brightening Toner 250ml","['marketplace', 'health & beauty', 'skincare', 'face cleansing']",None,None,neutral,neutral
1000383163032,"Raid Wasp, Mosquito Fly Killer with Eucalyptus Aroma Oil Aerosol Spray 300ml",http://groceries.asda.com/product/cleaning/raid-wasp-mosquito--fly-killer-with-eucalyptus-aroma-oil-aerosol-spray-ml/1000383163032,"laundry & household, cleaning, household essentials, insect sprays & pest control",2025-02-12,4.50,£15.00/lt,NaN,NaN,3,"Laundry & Household, Cleaning, Household Essentials, Insect Sprays & Pest Control - Raid Wasp, Mosquito Fly Killer with Eucalyptus Aroma Oil Aerosol Spray 300ml","laundry & household, cleaning, household essentials, insect sprays & pest control",None,None,neutral,neutral
111683060,Always Dailies Large Profresh Panty Liners,https://groceries.morrisons.com/products/always-dailies-large-profresh-panty-liners/111683060,toiletries & beauty > period products > liners > big packs,2025-02-12,3.40,NaN,NaN,NaN,4,Toiletries & Beauty > Period Products > Liners > Big Packs - Always Dailies Large Profresh Panty Liners,toiletries & beauty,period products,liners,n

In [208]:
df.shape

(25283, 15)

In [209]:
print(f"Classified as female: {sum(df["gender_breadcrumb"] == "female")/(df.shape[0]):.2%}")
print(f"Classified as male: {sum(df["gender_breadcrumb"] == "male")/(df.shape[0]):.2%}")
print(f"Classified as neutral: {sum(df["gender_breadcrumb"] == "neutral")/(df.shape[0]):.2%}")

Classified as female: 10.34%
Classified as male: 4.47%
Classified as neutral: 85.20%


## Use LLM to create a dictionary for ADM

In [210]:
# Gendered words dictionary for product packaging
gendered_words = {
    "feminine": [
        "gentle", "soft", "delicate", "radiant", "luminous", "soothing", "dewy",
        "floral", "elegance", "grace", "charm", "blossom", "rose", "pearl", "flower" , "nurturing", "nutritious"
        "for her", "designed for women", "feminine touch" , "volume" , "voluminous", "young" 
    ],
    "masculine": [
        "strong", "rugged", "bold", "powerful", "athletic", "robust", "tough",
        "strength", "force", "cedarwood", "musk", "steel", "for him",
        "engineered for men", "masculine scent", "beast", 
    ],
    "neutral": [
        "fresh", "clean", "natural", "eco-friendly", "sustainable", "modern",
        "quality", "care", "innovation", "balance", "nature"
    ]
}


In [211]:
female_keywords = {
    "female": {
        "food_beverages": [
            "guilt-free", "nourishing", "wholesome", "superfood", "antioxidant-rich",
            "low-calorie", "dairy-free", "revitalizing", "revitilize", "revitilise","revitilising", "skin-boosting", "detoxifying",
            "pure", "farm-to-table", "probiotic", "vitamin-packed", "heart-healthy",
            "indulgent", "crave-worthy", "mood-enhancing", "meal-prep-friendly"
        ],
        "clothing_apparel": [
            "flattering", "figure", "hugging", "chic", "feminine", "tailored",
            "confidence-boosting", "soft-to-the-touch", "curve-loving", "luxe",
            "couture-inspired", "occasion-ready", "wrinkle-resistant", "seasonless",
            "trend-forward", "size-inclusive"
        ],
        "beauty_skincare": [
            "hydrating", 
            "radiant", 
            "rejuvenating", 
            "age-defying", 
            "anti-aging",
            "glow-boosting", 
            "non-comedogenic", 
            "beauty", 
            "paraben-free",
            "sulfate-free", 
            "luminous", 
            "plumping", 
            "firming", 
            "brightening", 
            "plumed", 
            "hug", 
            "moisturizing", 
            "moisture", 
            "glitter", 
            "sensitive", 
            "pore-minimizing", 
            "pore-minimising",
            "spa-like", 
            "silky", 
            "velvety", 
            "velvet",
            "smudge-proof", 
            "pamper", 
            "young", 
            "elegant", 
            "elegance", 
            "pampering", 
            "clinically proven", 
            "sensitive-skin-friendly", 
            "revitalizing", 
            "revitilize", 
            "revitilise",
            "revitilising", 
            "miracle",
            "miracolous", 
            "frizzy",
            "luxuriously",
            "luxurious",
            "luxury",
            "sleek",
            "smoothness",
            "gently",
            "restore",
            "restoring",
            "macadamia",
            "moist",
            "moisturizes",
            "moisturises",
            "gentle", 
            "soft", 
            "delicate", 
            "radiant", 
            "luminous", 
            "soothing", 
            "dewy", 
            "floral", 
            "elegance", 
            "grace", 
            "charm", 
            "blossom", 
            "rose", 
            "pearl", 
            "flower", 
            "sensational",
            "sensual",
            "nurturing", 
            "nutritious",
            "her",
            "hers", 
            "designed for women", 
            "feminine touch" , 
            "volume" , 
            "voluminous", 
            "spoil", 
            "nourishing", 
            "avocado oil", 
            "radiating", 
            "smooth", 
            "coloured", 
            "colour", 
            "coloured-hair", 
            "shine",             
            "vibrancy", 
            "restore", 
            "restoring", 
            "regenerate", 
            "haircare", 
            "care", 
            "skincare", 
            "harmony", 
            "feel", 
            "healthy", 
            "vibrant", 
            "feminine",
            "women",
            "woman",
            "woman's",
            "women's"
        ],
        "cross_category": [
            "empowering", "curated", "personalized", "transformative", "mindful",
            "bespoke", "award-winning", "iconic", "must-have", "essentials",
            "perfect for you", "perfect for everyday"
        ]
    }
}

male_keywords = {
    "male": {
        "food_beverages": [
            "muscle-fuel", "keto-friendly", "low-carb", "energy-packed", "grass-fed",
            "no-added-sugar", "hearty", "bold flavors", "crafted", "smoked",
            "slow-cooked", "testosterone-supporting", "crunchy", "grillable",
            "fuel-for-the-day"
        ],
        "clothing_apparel": [
            "rugged", "moisture-wicking", "quick-dry", "athletic-fit", "slim-fit",
            "classic-cut", "performance-ready", "stretch-flex", "reinforced",
            "heavy-duty", "weather-resistant", "odour-resistant", "anti-microbial",
            "workwear-inspired", "built-to-last", "multi-pocket", "tech-friendly"
        ],
        "beauty_grooming": [
            "barber-approved", 
            "beard-friendly", 
            "anti-itch", 
            "long-lasting hold",
            "oil-free", 
            "matte finish", 
            "exfoliating", 
            "charcoal-infused",
            "post-shave", 
            "fade-resistant", "strengthening", 
            "travel-sized",
            "tattoo-safe", 
            "sweat-proof", 
            "protect",
            "ice",
            "iced",
            "zesty",
            "citrus",
            "chest",
            "irresistible",
            "sweat",
            "fight",
            "kill",
            "men's",
            "man's",
            "bold", 
            "no-nonsense", 
            "masculine", 
            "streamlined", 
            "modern",
            "go-anywhere", 
            "built-tough", 
            "precision-engineered",
            "mission-critical", 
            "uncompromising", 
            "lean",
            "attack",
            "strengthen",
            "stronger",
            "strongest",
            "bulldog",
            "technology",
            "odour-control",
            "odor-control",
            "masculine",
            "balanced",
            "engineered",
            "engineering",
            "confidence",
            "odor",
            "odour",
            "his",
            "him",
            "he",
            "men",
            "male",
            "man",
            "father",
            "son",
            "yourself",
            "be yourself",
            
            
            
        ]
    }
}

neutral_keywords = {
    "neutral": {
        "food_beverages": [
            "sustainably sourced", "organic", "plant-based", "non-GMO", "low-waste",
            "nutrient-dense", "ethically produced", "allergy-friendly", "gluten-free",
            "vegan", "fair-trade", "minimally processed", "whole-food", "balanced",
            "energy-boosting", "immune-supporting", "hydrating", "seasonal",
            "locally grown", "recyclable packaging", "zero-added-sugar", "flexitarian",
            "climate-friendly", "compostable", "mindfully crafted"
        ],
        "clothing_apparel": [
            "unisex", "size-inclusive", "eco-friendly", "versatile", "durable",
            "breathable", "easy-care", "timeless", "minimalist", "adjustable",
            "lightweight", "recycled materials", "modular", "gender-neutral fit",
            "functional", "adaptive", "travel-ready", "certified sustainable",
            "biodegradable", "packable", "long-lasting"
        ],
        "beauty_personal_care": [
            "gentle", 
            "hypoallergenic", 
            "fragrance-free", 
            "cruelty-free", 
            "reef-safe",
            "non-toxic", 
            "pH-balanced", 
            "zero-waste", 
            "dermatologist-tested",
            "sensitive-skin safe", 
            "refillable", 
            "waterless", 
            "carbon-neutral",
            "all-in-one", 
            "travel-friendly", 
            "plastic-free", 
            "universal",
            "transparent ingredients", 
            "ethically sourced", 
            "compostable packaging", 
            "clean",

            
            
        ],
        "cross_category": [
            "inclusive", "sustainable", "ethical", "innovative", "accessible",
            "adaptable", "transparent", "holistic", "universal", "eco-conscious",
            "future-proof", "community-driven", "thoughtfully designed",
            "quality-crafted", "multi-functional", "user-friendly", "planet-positive",
            "socially responsible"
        ]
    }
}

__Potential challenges__

1. How do we standardize prices to put it on the same scale?

    * option 1: create a categorical variable capturing the type of pricing -- but then how do you convert it into standardized df?

    * option 2 :bring to common denominator as much as possible


2. How do we study the gender price gap?

    * we need to find a way to assign the products into a category (female/male/neither)

3. What method do we study this with?

    * Moshary et al. 2023 use regression


__Pre-processing. Things to do__

- transform data to datetime object


Use one store info to categorize into female/male (Cyril's idea)